In [1]:
import os, sys, json
autocog_home = os.getenv("AUTOCOG_HOME", '/workspace/projects/autocog')
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.automatons.sta.automaton import StructuredThoughtAutomaton as STA
arch = CogArch()

In [9]:
arch.load(tag='test', language='sta', program="""
entry(formulate):

formats:
- sentence: 

prompt(formulate): 
- target(question) source(?question)
- target(comments) source(reflexion.comments)
> question(text): 
> comments[3](thought): 
> work[10](thought): 
> answer(sentence): 
__next(reflexion):

prompt(reflexion): 
- target(question) source(?question)
- target(hints) source(formulate.comments)
- target(answer) source(formulate.answer)
> question(text): 
> hints[3](thought): 
> answer(sentence):  
> work[10](thought):
> comments[3](thought):
__next(formulate,submit):

prompt(submit): 
- target(answer) source(reflexion.answer)
> answer(sentence):  
__exit(answer): 
""").model_dump()

{'tag': 'test',
 'orchestrator': {'frames': [{'pid': None,
    'ctag': None,
    'stacks': None,
    'subs': [1]},
   {'pid': 0, 'ctag': 'test', 'stacks': None, 'subs': []}],
  'LMs': {},
  'pipe': None,
  'limit': None,
  'cogs': {'test': {'tag': 'test'}}},
 'formats': {'record': {'label': 'record',
   'desc': 'start of a nested prompt',
   'caster': None},
  'text': {'label': 'text', 'desc': 'ASCII text in any form', 'caster': None},
  'thought': {'label': 'thought',
   'desc': 'your thoughts (a few words per lines)',
   'caster': None},
  'bool': {'label': 'bool',
   'desc': 'a boolean value use either True/False or yes/no',
   'caster': <function autocog.automatons.sta.automaton.StructuredThoughtAutomaton.__compile_formats.<locals>.<lambda>(x)>},
  'sentence': {'label': 'sentence', 'desc': '', 'caster': None},
  'formulate.next': {'label': 'next', 'desc': '', 'caster': None},
  'reflexion.next': {'label': 'next', 'desc': '', 'caster': None},
  'submit.next': {'label': 'next', 'desc

In [10]:
await arch('test', question='test')

AttributeError: 'Channel' object has no attribute 'prompt'